## Automatic differentiation and gradient tape

### Gradient tapes

Tensorflow 使用的是逆序自动微分方法来自动求得梯度的。在实现使用的是 Gradient Tape 算法，而不是 Naive 的树。

TF 的 `tf.GradientTape` API 提供梯度的自动计算。TF 会将所有操作记录到 `tf.GradientTape` 上，然后通过逆序自动微分方法来求得导数。

In [1]:
import tensorflow as tf

In [5]:
x = tf.ones((2, 2))

with tf.GradientTape() as t:
    t.watch(x) # 输入变量作为根结点
    y = tf.reduce_sum(x)
    z = tf.multiply(y, y)
    print(y)
    print(z)

tf.Tensor(4.0, shape=(), dtype=float32)
tf.Tensor(16.0, shape=(), dtype=float32)


In [6]:
dz_dx = t.gradient(z, x)

for i in [0, 1]:
    for j in [0, 1]:
        print(dz_dx[i][j])

tf.Tensor(8.0, shape=(), dtype=float32)
tf.Tensor(8.0, shape=(), dtype=float32)
tf.Tensor(8.0, shape=(), dtype=float32)
tf.Tensor(8.0, shape=(), dtype=float32)


#### `tf.GradientTape`  类

`tf.GradientTape` 用于记录操作实现自动微分。

在实现自动微分时，输入变量是被当作根节点的，我们需要对输入变量进行 `watch` 创建一个上下文环境。

当我们使用 `tf.Varaiable` 创建一个变量时，如果指定了第二个参数  `trainable=True` 那么该变量会自动被 watched.


In [11]:
x = tf.Variable(3.0)

with tf.GradientTape() as t:
    t.watch(x)
    y = x * x

dy_dx = t.gradient(y, x)

# y = x^2, dy_dx = 2x
print(dy_dx)


tf.Tensor(6.0, shape=(), dtype=float32)


#### 高阶积分

通过对 GradientTapes 的嵌套，我们可以计算高阶的导数。如：

In [17]:
x = tf.constant(3.0)
with tf.GradientTape() as g:
    g.watch(x)
    with tf.GradientTape() as gg:
        gg.watch(x)
        y = x * x

    dy_dx = gg.gradient(y, x) 
dy_dx2 = g.gradient(dy_dx, x)

# y = x^2, y' = 2x, y'' = 2

print(dy_dx)
print(dy_dx2)

tf.Tensor(6.0, shape=(), dtype=float32)
tf.Tensor(2.0, shape=(), dtype=float32)


默认情况下，当调用了 `gradient()` 方法后，GradientType 对象就会被释放，为了多次计算，我们可以对该对象进行持久化，当计算完成后，删除该对象即可。

In [22]:
x = tf.constant(3.0)
with tf.GradientTape(persistent=True) as g:
    g.watch(x)
    y = x * x
    z = y * y
dz_dx = g.gradient(z, x)
dy_dx = g.gradient(y, x)
del g

# z = x^4, z' = 4*x^3
print(dz_dx)
# y = x^2, y' = 2*x
print(dy_dx)


tf.Tensor(108.0, shape=(), dtype=float32)
tf.Tensor(6.0, shape=(), dtype=float32)


现在再用一个 $z = xy + sin(x)$ 的例子来说明下。

In [41]:
x = tf.Variable(0.5)
y = tf.Variable(4.5)

with tf.GradientTape(persistent=True) as t:
    t.watch(x)
    z = x * y + tf.math.cos(x) #  为什么要用 tf.math.cos, 因为 tf 的操作才会记录操作。

dz_dx = t.gradient(z, x)
dz_dy = t.gradient(z, y)
print(dz_dx)
print(dz_dy)

del t

tf.Tensor(4.0205746, shape=(), dtype=float32)
tf.Tensor(0.5, shape=(), dtype=float32)


当在参与的运算中有一个变量被 watch， 那么参与运算的所有变量都会被 watch。我们可以通过 `watch_accessed_variables=False` 来取消这个配置。

In [26]:
a = tf.Variable(3.0)
b = tf.Variable(4.0)
with tf.GradientTape(watch_accessed_variables=False) as tape:
  tape.watch(a)
  y = a ** 2  # Gradients will be available for `variable_a`.
  z = b ** 3  # No gradients will be available since `variable_b` is
                       # not being watched.

**注意**，在模型中对层的参数进行 watch 的时候，要保证层已经调用了 build。否则了进行 watch 时并且应用了  `watch_accessed_variables=False`, 那很可能得不到任何梯度。所以如果不是性能问题，一般不要置成 `False`.


In [40]:
inputs = tf.ones((32,1))
a = tf.keras.layers.Dense(32, input_shape=(32,1))
b = tf.keras.layers.Dense(32)

with tf.GradientTape(watch_accessed_variables=False) as tape:
  tape.watch(a.variables)  # Since `a.build` has not been called at this point
                           # `a.variables` will return an empty list and the
                           # tape will not be watching anything.
  result = b(a(inputs))
  g = tape.gradient(result, a.variables)  # The result of this computation will be
                                      # a list of `None`s since a's variables
                                      # are not being watched.
print(g)

[None, None]
